In [10]:

from pathlib import Path
import torch
torch.cuda.empty_cache()
paths = [str(x) for x in Path('C:/langchain2/wiki/wiki_2/trext_data').glob('**/*.txt')]
paths

['C:\\langchain2\\wiki\\wiki_2\\trext_data\\Chikungunya.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\dengue.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Fatigue.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Fever.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Headache.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Malaria.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Nausea.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Rash.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Tuberculosis.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Typhoid_fever.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Vomiting.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\who_dengue.txt']

In [11]:
# initialize an empty string to hold all the text
text = ""

# loop over the file paths and read each file into the text string
for path in paths:
    with open(path, 'r', encoding='utf-8') as f:
        text += f.read()


In [12]:
text = text.replace('\n', ' ')

In [13]:
examples=text

In [14]:
print("Length of text: {} characters".format(len(text)))

Length of text: 310883 characters


In [15]:
# # split the text into smaller chunks
# chunks = [text[i:i+100] for i in range(0, len(text), 100)]
# num_chunks = len(chunks)
# print("Number of chunks created:", num_chunks)

In [16]:
from transformers import AutoTokenizer
import torch 

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [17]:
def preprocess_function(examples):
    return tokenizer(examples, padding=True, truncation=True)


In [18]:
tokenized_eli5 = examples.map(
    preprocess_function,
    batched=True,
    num_proc=4
)

AttributeError: 'str' object has no attribute 'map'

In [ ]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15).map(group_texts, batched=True, num_proc=4)

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_mlm_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()